# Import libraries

In [18]:
import pandas as pd
import numpy as np

import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import linear_kernel

# Reading Data

In [3]:
df1 = pd.read_csv('dataset/tarih.csv')
df1

,question,answer
0,А Байтұрсынұлының 1909 жылы қамалғанға дейінгі...,Қарқаралыда оқытушы
1,А Байтұрсынов пен М Сералин өзі жазбаларында Қ...,Қостанай
2,А Байтұрсыновтың 1911 жылы Орынборда жарық көр...,″Маса″
3,А Байтұрсыновтың Орынборда жарық көрген жинағы –,″Маса″
4,А Байтұрсыновтың өлеңдер жинағы:,«Маса»
...,...,...
15135,XX қозғалыс кезінде және жазалау нәтижесінде Қ...,Жарты миллион
15136,XX съезден кейін ақталған қайраткер:,Н Нұрмақов
15137,XX съезден кейін ақталған қайраткер:,С Асфендияров
15138,XXI ғасырдың басында Республикадағы мектептер...,8 мыңнан астам


In [4]:
def swap(df):
    new_df = pd.DataFrame(df.values.tolist(), columns = ['question', 'answer'])
    que = new_df.question.tolist()
    new_df.question = new_df.answer.tolist()
    new_df.answer = que
    return new_df

#swap_df = swap(df1)

#df = pd.concat([df1, swap_df], ignore_index = True)
df = pd.concat([df1], ignore_index = True)
df

,question,answer
0,А Байтұрсынұлының 1909 жылы қамалғанға дейінгі...,Қарқаралыда оқытушы
1,А Байтұрсынов пен М Сералин өзі жазбаларында Қ...,Қостанай
2,А Байтұрсыновтың 1911 жылы Орынборда жарық көр...,″Маса″
3,А Байтұрсыновтың Орынборда жарық көрген жинағы –,″Маса″
4,А Байтұрсыновтың өлеңдер жинағы:,«Маса»
...,...,...
15135,XX қозғалыс кезінде және жазалау нәтижесінде Қ...,Жарты миллион
15136,XX съезден кейін ақталған қайраткер:,Н Нұрмақов
15137,XX съезден кейін ақталған қайраткер:,С Асфендияров
15138,XXI ғасырдың басында Республикадағы мектептер...,8 мыңнан астам


# Cleaning

In [5]:
def clean_str(text):
    stripped = re.sub('[^0-9,^a-zA-Z, ^А-Я,а-я,Ә,І,Ң,Ғ,Ү,Ұ,Қ,Ө,Һ,ә,і,ə,ң,ғ,ү,ұ,қ,ө,һ]', ' ', str(text).replace('-', ' '))
    stripped = re.sub('_', '', stripped)
    stripped = re.sub('\s+', ' ', stripped)
    return stripped.replace(',','')
    

def cleaning_data(code_data):
    question = code_data.question.tolist()
    answer = code_data.answer.tolist()
    sentences = []
    for line in range(len(question)):
        sentences.append([str(clean_str(question[line])).lower(), 
                          str(clean_str(answer[line])).lower()])
    return pd.DataFrame(sentences, columns = ['question', 'answer'])

In [6]:
df = cleaning_data(df)
df

,question,answer
0,а байтұрсынұлының 1909 жылы қамалғанға дейінгі...,қарқаралыда оқытушы
1,а байтұрсынов пен м сералин өзі жазбаларында қ...,қостанай
2,а байтұрсыновтың 1911 жылы орынборда жарық көр...,маса
3,а байтұрсыновтың орынборда жарық көрген жинағы,маса
4,а байтұрсыновтың өлеңдер жинағы,маса
...,...,...
15135,xx қозғалыс кезінде және жазалау нәтижесінде қ...,жарты миллион
15136,xx съезден кейін ақталған қайраткер,н нұрмақов
15137,xx съезден кейін ақталған қайраткер,с асфендияров
15138,xxi ғасырдың басында республикадағы мектептер...,8 мыңнан астам


In [7]:
le = LabelEncoder()

In [8]:
df['label'] = le.fit_transform(df['answer'])
df

,question,answer,label
0,а байтұрсынұлының 1909 жылы қамалғанға дейінгі...,қарқаралыда оқытушы,10413
1,а байтұрсынов пен м сералин өзі жазбаларында қ...,қостанай,10526
2,а байтұрсыновтың 1911 жылы орынборда жарық көр...,маса,935
3,а байтұрсыновтың орынборда жарық көрген жинағы,маса,935
4,а байтұрсыновтың өлеңдер жинағы,маса,935
...,...,...,...
15135,xx қозғалыс кезінде және жазалау нәтижесінде қ...,жарты миллион,6551
15136,xx съезден кейін ақталған қайраткер,н нұрмақов,7744
15137,xx съезден кейін ақталған қайраткер,с асфендияров,1103
15138,xxi ғасырдың басында республикадағы мектептер...,8 мыңнан астам,445


In [9]:
df['label'].max()

11154

# Naive Bayes

In [11]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(df.question)

In [33]:
print(X.toarray()[0:5])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [32]:
X.shape

(15140, 17988)

In [57]:
import pickle

In [66]:
pickle.dump(vectorizer, open('_model.sav', 'wb'))
pickle.dump(X, open('_vectorized.sav', 'wb'))

In [44]:
cosine_similarities = linear_kernel(vectorizer.transform(['xx кезінде және жазалау нәтижесінде қазақстанның жеріндегі қазақтардың саны кеміді']), X).flatten()
cosine_similarities = np.ndarray.argsort(-cosine_similarities)[:10]
df.loc[cosine_similarities.tolist(), ['question', 'answer']]

,question,answer
12213,қазақ кср і,1936 1991 ж
11740,қазақ акср і,1920 1936 ж
3610,қазақ акср і құрылды,1920 жылы 4 қазанда орынборда
3609,қазақ акср і құрылды,1920 жылы сәуірде
3946,қазақ ханы жәнібек,ақ орда ханы барақтың баласы
3608,қазақ акср і қазақ кеңестік социалистік респуб...,1936 жылы
3687,қазақ жүздері,ортажүз кіші жүз ұлы жүз
3881,қазақ хандығында жүз,бірнеше ұлыстан тұрды
3906,қазақ хандығының алғашқы ханы,керей
9232,1643 жылы қазақ жоңғар ұрысында қазақ әскерін...,жәңгір хан


In [45]:
cosine_similarities = linear_kernel(vectorizer.transform(['xx қазақ жеріндегі кезінде және жазалау нәтижесінде қазақстанның қазақтардың саны кеміді']), X).flatten()
cosine_similarities = -np.sort(-cosine_similarities)[:10]
cosine_similarities

array([0.92613209, 0.74220452, 0.7248068 , 0.23696139, 0.21535982,
       0.20474668, 0.18307976, 0.18007796, 0.17175527, 0.16818403])

In [45]:
df.question[15135:15136].tolist()

['xx қозғалыс кезінде және жазалау нәтижесінде қазақстанның жеріндегі қазақтардың саны кеміді ']

In [46]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = linear_kernel(vectorizer.transform(['xx қозғалыс кезінде және жазалау нәтижесінде қазақстанның жеріндегі қазақтардың саны кеміді']), X).flatten()
cosine_similarities = np.ndarray.argsort(-cosine_similarities)[:10]
df.loc[cosine_similarities.tolist(),'answer']

15135                            жарты миллион
11154                     жарты миллионға жуық
11205                    жарты миллионға жуық 
11290     иттифок и муслимин партиясы құрылды 
13560      қазақстанның демократиялық таңдауы 
11246                қырғыз ұйғыр шаруаларымен
11187                 қырғыз ұйғыр шаруалармен
697                              невада семей 
12288                                  375 мың
4375                                     азат 
Name: answer, dtype: object

In [26]:
cosine_similarities = np.ndarray.argsort(-cosine_similarities)[:10]
cosine_similarities

array([15137, 15136,  7313,  3302,  3292,  3294, 15060, 15039, 14971,
       14970], dtype=int64)

In [32]:
cosine_similarities.tolist()

[15137, 15136, 7313, 3302, 3292, 3294, 15060, 15039, 14971, 14970]

In [41]:
df.loc[cosine_similarities.tolist(),'answer']

15137                                        с асфендияров
15136                                           н нұрмақов
7313     сейфуллин майлин жансүгіров асфендияров жандос...
3302                             с асфендияров о жандосов 
3292                     с сейфуллин і жансүгіров б майлин
3294                    с сейфуллин і жансүғіров б майлин 
15060                          м жұмабаев  а байтұрсынұлы 
15039                                           и в сталин
14971                                           м жұмабаев
14970    м жұмабаев  ш құдайбердіұлы  ж аймауытов  а ба...
Name: answer, dtype: object

In [23]:
for i in cosine_similarities:
    print(df.question.tolist()[i])

 xx съезден кейін ақталған қайраткер 
xx съезден кейін ақталған қайраткер 
сталин өлімінен кейін хх съезден кейін ақталғандар 
кокп нің xx сьезінен кейін ақталған мемлекет қайраткерлері 
кокп xx съезінен кейін ақталған аса көрнекті әдебиетшілер 
кокп xx сьезінен кейін ақталған әділетсіз жазаға ұшыраған аса көрнекті әдебиетшілер 
 xx ғасырдың 80 жылдары шығармашылық мұралары ақталған қайраткерлер 
xx ғасырдың 30 жылдары тоталитарлық казармалық жүйені енгізген қайраткер 
xx ғ 80 ж соңында шығармашылық мұралары ақталған қайраткердің бірі 
xx ғ 80 ж соңында шығармашылық мұралары ақталған қайраткердің бірі 


In [11]:
X.shape

(15140, 8618)

In [12]:
from sklearn.decomposition import PCA
pca = PCA(500) 
pca.fit(X.toarray())

PCA(copy=True, iterated_power='auto', n_components=500, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [13]:
pca_data = pca.transform(X.toarray())
pca_data.shape

(15140, 500)

In [17]:
pca_data

array([[-1.24970130e-02, -5.71863915e-03, -4.80605722e-03, ...,
         3.84977890e-04,  2.99237905e-04,  4.40740719e-04],
       [-1.32837702e-02, -6.08416226e-03, -5.57754288e-03, ...,
         4.92850450e-02,  6.84579601e-03, -2.91505141e-03],
       [-1.32768178e-02, -6.17421190e-03, -5.25985653e-03, ...,
         1.23971260e-02, -3.72135482e-02,  7.75626312e-04],
       ...,
       [-1.39015362e-02, -6.54955253e-03, -5.64278339e-03, ...,
         4.26856416e-05, -4.22669236e-04, -1.29840285e-04],
       [-1.80585194e-02, -1.00201753e-02, -7.69776673e-03, ...,
        -8.63402469e-03, -9.00070930e-03,  5.67860524e-02],
       [-1.24970130e-02, -5.71863915e-03, -4.80605722e-03, ...,
         3.84177762e-04,  3.01904299e-04,  4.40739274e-04]])

In [19]:
from sklearn import svm
naive_bayes = svm.SVC(kernel='linear')
naive_bayes.fit(pca_data, y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [26]:
#x_test = df.question[Train_split:].tolist()
#y_test = df.label[Train_split:].tolist()

predictions = naive_bayes.predict(pca.transform(X[0:10].toarray()))

print('Accuracy score: {}'.format(accuracy_score(y[0:10], predictions)))

Accuracy score: 0.0


In [16]:
le.inverse_transform(naive_bayes.predict(vectorizer.transform(['ыстық көл']).toarray()))

array(['1856 ж ш уалиханов топографиялық картаға түсірген көл '],
      dtype=object)

In [17]:
naive_bayes.predict(vectorizer.transform(['ыстық көл']).toarray())

array([3881])

In [18]:
df[df.label == 3881]

,question,answer,label
2450,ыстық көл,1856 ж ш уалиханов топографиялық картаға түсір...,3881


In [16]:
print(df.question.tolist()[401])
print(df.answer.tolist()[401])

ыстық көл жағасында 
үйсіндердің астанасы чигучен қызыл аңғар қаласы орналасқан 


In [27]:
le.inverse_transform(naive_bayes.predict(vectorizer.transform(["жоңғар аңырақай шайқасы"]).toarray()))

array(['1718 ж '], dtype=object)